In [5]:
from keras_tuner import Hyperband
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split

In [6]:
# Load the CIFAR-10 dataset
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Split the training set for validation during hyperparameter tuning
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [7]:
def build_tunable_model(hp):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(1024, activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    optimizer_name = hp.Choice("optimizer", ["adam", "sgd", "rmsprop"])
    lr = hp.Float("lr", min_value=1e-5, max_value=1e-1, sampling="log")
    
    # Choose the optimizer
    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
tuner = Hyperband(
    build_tunable_model,
    objective="val_accuracy",
    max_epochs=10,
    directory="hyperparameter_tuning",
    project_name="learning_rate_tuning",
)

In [9]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Trial 30 Complete [00h 01m 40s]
val_accuracy: 0.7041000127792358

Best val_accuracy So Far: 0.7271000146865845
Total elapsed time: 00h 22m 31s


In [10]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best learning rate: {best_hps.get('lr')}")
print(f"Best optimizer: {best_hps.get('optimizer')}")

Best learning rate: 0.0007068016444742155
Best optimizer: rmsprop


In [11]:
final_model = tuner.hypermodel.build(best_hps)
history = final_model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test), batch_size=32)

Epoch 1/20
1250/1250 [==============================] - 14s 10ms/step - loss: 1.3223 - accuracy: 0.5411 - val_loss: 1.7489 - val_accuracy: 0.4863
Epoch 2/20
1250/1250 [==============================] - 12s 10ms/step - loss: 0.9406 - accuracy: 0.6744 - val_loss: 1.2272 - val_accuracy: 0.5940
Epoch 3/20
1250/1250 [==============================] - 12s 10ms/step - loss: 0.7617 - accuracy: 0.7365 - val_loss: 1.2008 - val_accuracy: 0.6188
Epoch 4/20
1250/1250 [==============================] - 12s 10ms/step - loss: 0.6227 - accuracy: 0.7874 - val_loss: 1.1433 - val_accuracy: 0.6653
Epoch 5/20
1250/1250 [==============================] - 13s 10ms/step - loss: 0.5104 - accuracy: 0.8249 - val_loss: 1.3943 - val_accuracy: 0.6742
Epoch 6/20
1250/1250 [==============================] - 14s 11ms/step - loss: 0.4081 - accuracy: 0.8580 - val_loss: 1.1353 - val_accuracy: 0.6943
Epoch 7/20
1250/1250 [==============================] - 12s 10ms/step - loss: 0.3375 - accuracy: 0.8817 - val_loss: 1.4566 -

In [12]:
final_loss, final_accuracy = final_model.evaluate(x_test, y_test)
print(f"Final Test Loss: {final_loss}")
print(f"Final Test Accuracy: {final_accuracy}")

313/313 [==============================] - 1s 5ms/step - loss: 2.1699 - accuracy: 0.7029
Final Test Loss: 2.169945478439331
Final Test Accuracy: 0.7028999924659729
